In [3]:
import tkinter as tk
from tkinter import filedialog
import csv
import os
import shutil
###################################################################
######## Root window functions
def open_form():
    global form_path
    form_path = filedialog.askopenfilename(filetypes= [('.csv file','*.csv')])
    form_entry.delete(0, tk.END)
    form_entry.insert(0, form_path)
def open_files():
    global file_paths
    file_paths = filedialog.askopenfilenames(filetypes= [('.txt file','*.txt')])
    path_out = ''
    for i in range(len(file_paths)):
        path_out = path_out+str(i+1)+'. '+file_paths[i]+'\n'
    files_entry.delete('1.0', tk.END)
    files_entry.insert('1.0', path_out)
def rename_go():
    global form_path, file_paths
    form = []
    with open(form_path, 'r') as f:
        for row in f:
            row = row.strip('\n ').rstrip(',').split(',')
            if row != ['']:
                form.append(row)
    for i in range(1,len(form)):
        for j in range(1,len(form[0])):
            fbc = form[0][j].split(':')[-1].strip(' ').upper()
            rbc = form[i][0].split(':')[-1].strip(' ').upper()
            for file in file_paths:
                if fbc in file and rbc in file:
                    file = os.path.normpath(file)
                    folder = file.replace(file.split(os.sep)[-1],form[i][j])
                    try:
                        os.mkdir(folder)
                    except:
                        shutil.rmtree(folder)
                        os.mkdir(folder)
                    new_name = folder+os.sep+file.split(os.sep)[-1].strip('.txt')+'√.txt'
                    os.renames(file,new_name)
    rename_state.config(text = 'Files Renamed.')
###################################################################
######## Preview window functions
def preview():
    global file_paths, form_path
    data = []
    with open(form_path, 'r') as f:
        for row in f:
            row = row.strip('\n ').rstrip(',').split(',')
            if row != ['']:
                data.append(row)
    create_entry_window(data)
def create_entry_window(data):
    global file_paths
    file_count = len(file_paths)
    entry_window = tk.Toplevel(root)
    entry_window.title("Preview renaming form - S3 File Renaming Tool")
    epn_label = tk.Label(entry_window, text = 'Colour Explanation: ')
    epn_label.grid(row = 0, column = 0, columnspan = 2, padx=4, pady=2)
    epn_entry = tk.Entry(entry_window, width = 15, bg = 'LightBlue')
    epn_entry.grid(row = 0, column = 2, padx=4, pady=2)
    epn_entry.insert(0,'Barcodes')
    epn2_entry = tk.Entry(entry_window, width = 15, bg = 'LightPink')
    epn2_entry.grid(row = 0, column = 3, padx=4, pady=2)
    epn2_entry.insert(0,'Files found')
    epn3_entry = tk.Entry(entry_window, width = 15)
    epn3_entry.grid(row = 0, column = 4, padx=4, pady=2)
    epn3_entry.insert(0,'Files not found')
    global entry_list,save_state
    entry_list = []
    sub = []
    count = 0
    for i in range(len(data)):
        for j in range(len(data[i])):
            if i == 0 and j == 0:
                entry = tk.Entry(entry_window, width = 15)
                entry.config(state = 'disabled')
            elif i == 0 or j == 0:
                entry = tk.Entry(entry_window, width = 15, bg="LightBlue")
            elif checkbc(data,i,j,file_paths) == True:
                entry = tk.Entry(entry_window, width = 15, bg="LightPink")
                count += 1
            else:
                entry = tk.Entry(entry_window, width = 15)
            entry.grid(row=i+2, column=j, padx=4, pady=3)
            sub.append(entry)
            entry.insert(0, data[i][j])
        entry_list.append(sub)
        sub = []
    count_lbl = tk.Label(entry_window, text = str(count)+'/'+str(file_count)+' selected files matched on the renaming form')
    count_lbl.grid(row = 1, column = 1, columnspan = 5, padx=4, pady=5)
    save_state = tk.Label(entry_window, text = '')
    save_state.grid(row = i+3, column = 3, padx=4, pady=10)
    save_button = tk.Button(entry_window, text='     Save changes in the renaming form     ',command = save_form)
    save_button.grid(row = i+3, column = 0, columnspan = 3, padx=4, pady=10)
    quit_button = tk.Button(entry_window, text='     Quit preview     ',command = entry_window.destroy)
    quit_button.grid(row = i+3, column = 5, columnspan = 2, padx=4, pady=10)
def save_form():
    global entry_list,save_state
    new_data = []
    sub = []
    for row in entry_list:
        for entry in row:
            sub.append(entry.get())
        new_data.append(sub)
        sub = []
    csvfile = open(form_path,'w',newline='')
    write = csv.writer(csvfile)
    write.writerows(new_data)
    csvfile.close()
    save_state.config(text="Changes saved.")
###################################################################
######## Functions
def checkbc(data,i,j,file_paths):
    fbc = data[0][j].split(':')[-1].strip(' ,').upper()
    rbc = data[i][0].split(':')[-1].strip(' ,').upper()
    rst = False
    for file in file_paths:
        if fbc in file and rbc in file:
            rst = True
            break
    return rst

#==================================================================
root = tk.Tk()
root.title("NGS peptide analysis - S3 File Renaming Tool")

form_button = tk.Button(root, text='      Select the renaming form    ',command = open_form)
form_button.grid(row = 0, column = 4, columnspan = 2, padx = 5, pady = 5)
form_entry = tk.Entry(root, width = 75)
form_entry.grid(row = 0, column = 0, columnspan = 4, padx = 5, pady = 5)

files_button = tk.Button(root, text="  Select the files to be renamed  ", command=open_files)
files_button.grid(row=1, column=4, rowspan = 3, columnspan=2, padx=5, pady=5)
files_entry = tk.Text(root,height=10, width=75)
files_entry.grid(row=1, column=0, rowspan = 3, columnspan=4, padx=5, pady=5)

preview_button = tk.Button(root, text = '   Preview and check   ',command = preview)
preview_button.grid(row = 4, column = 0, columnspan = 1, padx = 5, pady = 5)

rename_button = tk.Button(root, text = '      Rename (Caution: not reversible!)      ',command = rename_go)
rename_button.grid(row = 4, column = 1, columnspan = 3, padx = 5, pady = 5)

rename_state = tk.Label(root,text = '')
rename_state.grid(row = 4, column = 4, columnspan = 1, padx = 5, pady = 5)

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\86188\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-3-d064bb45875b>", line 36, in rename_go
    os.mkdir(folder)
FileExistsError: [WinError 183] 当文件已存在时，无法创建该文件。: 'C:\\Users\\86188\\Desktop\\error prone\\Example\\Example\\EcoRI-旧-2-phage'
